## 랭체인 기초
- 업스테이지 Solar Pro API 호출


In [3]:
%pip install openai langhchain langchain-core langchain-upstage langchain_text_splitters langchain-chroma

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langhchain (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for langhchain


In [2]:
%pip uninstall httpx
%pip install httpx==0.27.2

^C
Note: you may need to restart the kernel to use updated packages.
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# 1단계: client(모델 선택-solar)
from langchain_upstage import ChatUpstage

llm = ChatUpstage(api_key="up_2C93b7F2sI0B2JcSvw2qJuOQambdr", model="solar-pro2")
llm

ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x000002B4D5FC7AA0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002B4D64AEFF0>, model_name='solar-pro2', model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar')

In [5]:
# 2단계. 질문
query = input('질문을 입력하세요 : ')
llm.invoke(query)

AIMessage(content='저는 **Upstage**에서 개발한 대규모 언어 모델 **Solar Pro 2**입니다.  \n\n### 📌 **제 소개**  \n- **이름**: Solar Pro 2 (Thinking/Non-thinking 모드 지원)  \n- **개발사**: Upstage (2020년 설립, AI 기술 선도 기업)  \n- **모델 규모**: 30.9B 파라미터  \n- **지원 언어**: 한국어, 일본어, 영어 등  \n- **특징**:  \n  - **Thinking Mode**: 복잡한 추론/분석 가능 (하이브리드 모드)  \n  - **Non-thinking Mode**: 빠른 응답에 최적화  \n  - 문서 처리(**OCR**, **KIE**), 요약, 번역, 코드 생성 등 다양한 작업 수행  \n\n### 🚀 **주요 서비스**  \n- **Solar API**: 채팅, 구조화 출력, 함수 호출 등 개발자에게 유용한 기능 제공  \n- **Document AI**: PDF/이미지 문서의 텍스트 추출 및 키 정보 분석  \n- **온디바이스 AI**: 로컬 환경에서의 경량화 처리 가능  \n\n저는 AI와 인간의 효율적인 협업을 목표로 설계되었습니다. 😊  \n궁금한 점이 있다면 언제든 물어보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 231, 'prompt_tokens': 18, 'total_tokens': 249, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_na

## RAG

In [6]:
# 1단계. 문서를 읽는다.
from langchain_upstage import UpstageDocumentParseLoader

loader = UpstageDocumentParseLoader(
    file_path="2887010252_b3cO0MNJ_20251223041839.pdf",
    api_key="up_2C93b7F2sI0B2JcSvw2qJuOQambdr",
    output_format='html',
    coordinates=False
)

docs = loader.load()

docs

[Document(metadata={'total_pages': 23}, page_content='<p id=\'0\' data-category=\'paragraph\' style=\'font-size:14px\'>25-22(통권 1001호) 2025.12.19</p><p id=\'1\' data-category=\'paragraph\' style=\'font-size:18px\'>2026년 글로벌 트렌드</p><p id=\'2\' data-category=\'paragraph\' style=\'font-size:20px\'>목 차</p><p id=\'3\' data-category=\'paragraph\' style=\'font-size:14px\'>v</p><p id=\'4\' data-category=\'paragraph\' style=\'font-size:18px\'>■ 2026년 글로벌 트렌드</p><p id=\'5\' data-category=\'index\' style=\'font-size:16px\'>Executive Summary ········································································ ⅰ<br>1. 개요 ······························································································ 1<br>2. 2026년 글로벌 트렌드 ································································· 2</p><p id=\'6\' data-category=\'list\' style=\'font-size:16px\'>□ 비상업 목적으로 본 보고서에 있는 내용을 인용 또는 전재할 경우 내용의 출처를 명시<br>하면 자유롭게 인용할 수 있으며, 보고서 내용에 대한 문의는 아래와 같이 하여 주시<br>기 바랍니다.</p><br><p id=\'7\' dat

In [7]:
# 2단계. 문서를 나눈다 (chunking)

from langchain_text_splitters import (Language, RecursiveCharacterTextSplitter)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, # 1000자 단위로 쪼갠다.
    language=Language.HTML,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'total_pages': 23}, page_content="<p id='0' data-category='paragraph' style='font-size:14px'>25-22(통권 1001호) 2025.12.19</p><p id='1' data-category='paragraph' style='font-size:18px'>2026년 글로벌 트렌드</p><p id='2' data-category='paragraph' style='font-size:20px'>목 차</p><p id='3' data-category='paragraph' style='font-size:14px'>v</p><p id='4' data-category='paragraph' style='font-size:18px'>■ 2026년 글로벌 트렌드</p><p id='5' data-category='index' style='font-size:16px'>Executive Summary ········································································ ⅰ<br>1. 개요 ······························································································ 1<br>2. 2026년 글로벌 트렌드 ································································· 2</p><p id='6' data-category='list' style='font-size:16px'>□ 비상업 목적으로 본 보고서에 있는 내용을 인용 또는 전재할 경우 내용의 출처를 명시<br>하면 자유롭게 인용할 수 있으며, 보고서 내용에 대한 문의는 아래와 같이 하여 주시<br>기 바랍니다.</p><br>"),
 Document(metadata={'total_pages': 23}, page_content=

In [8]:
splits

[Document(metadata={'total_pages': 23}, page_content="<p id='0' data-category='paragraph' style='font-size:14px'>25-22(통권 1001호) 2025.12.19</p><p id='1' data-category='paragraph' style='font-size:18px'>2026년 글로벌 트렌드</p><p id='2' data-category='paragraph' style='font-size:20px'>목 차</p><p id='3' data-category='paragraph' style='font-size:14px'>v</p><p id='4' data-category='paragraph' style='font-size:18px'>■ 2026년 글로벌 트렌드</p><p id='5' data-category='index' style='font-size:16px'>Executive Summary ········································································ ⅰ<br>1. 개요 ······························································································ 1<br>2. 2026년 글로벌 트렌드 ································································· 2</p><p id='6' data-category='list' style='font-size:16px'>□ 비상업 목적으로 본 보고서에 있는 내용을 인용 또는 전재할 경우 내용의 출처를 명시<br>하면 자유롭게 인용할 수 있으며, 보고서 내용에 대한 문의는 아래와 같이 하여 주시<br>기 바랍니다.</p><br>"),
 Document(metadata={'total_pages': 23}, page_content=

In [9]:
# 3단계. 문서를 임베딩한다. 
# - 1000글자 내외로 나누어진 한글 문장을 숫자화 합니다. WHY? 벡터DB에 저장하기 위해서

from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(
    api_key="up_2C93b7F2sI0B2JcSvw2qJuOQambdr",
    model="solar-embedding-1-large-query"
)

In [10]:
# 4단계. 벡터DB에 임베딩 후 저장한다
# - 그럼 벡터DB가 필요하네? chroma => langhchain

from langchain_chroma import Chroma

vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_db"
)

vector_store.add_documents(splits)

['77df2455-35e5-4fe1-acac-21b7f936baa5',
 'e919504a-4103-457b-aa6c-14940a042350',
 '156825d0-d052-43af-afe5-b03591d84402',
 '79dce97a-113c-402e-8fd1-c3451f7b5471',
 '4ad15a2f-8c34-4289-8479-04b0a9e44265',
 '098c4eb0-fbed-438e-b490-6544073bbfe0',
 '80c9292c-2c93-4429-afe9-10488d1d76d4',
 '89aca7b6-0c7a-414b-8448-64375742fb96',
 '5c91590e-0356-4544-a440-f4456b66b200',
 '1ef5fe3a-bdd7-4865-99a3-8bd87907801a',
 '6761e1ab-7ad7-470b-9d26-9f61fe3abac0',
 '362f5824-d9b6-4049-bc79-7fe243ea7258',
 '837cf017-493e-41fb-966f-e3b05a1f279d',
 'e270b150-1b90-4fac-ac76-8e37146674f2',
 '47786acd-dbbe-44d6-8bca-8317b025d045',
 '2e73404b-1eeb-4fb4-82c9-f285df03ae8f',
 'b28a6795-9fb4-433d-af38-70b8d11aaf4d',
 '9359cffa-801c-42ab-92d7-9e489b2f0b4b',
 '3456d49d-5a40-4b93-a84f-e0759301d70d',
 '8bf67970-de28-4ace-a582-221e6ba828bf',
 '68b31d9b-3d69-43e8-b4dc-a2676ca6fa4d',
 '1ae595bf-604c-46f8-8902-f406cce5fb3a',
 'eaf2525f-0053-4d4f-b42e-3ff7e4e6aa1b',
 '8e7dc139-4b79-4570-a496-ef5b4b888e53',
 '9c4c0ac3-3a48-

In [11]:
# 2-1단계. Retriever(VectorDB에서 비슷한 문서를 찾는 작업)

retriever = vector_store.as_retriever()

retriever

VectorStoreRetriever(tags=['Chroma', 'UpstageEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002B4D65A9370>, search_kwargs={})

In [12]:
retriever.invoke("요즘엔 어떤 종목에 투자하는 것이 좋을까?")

# 지식을 AI가 잘 이해할 수 있도록 돕는 작업 (사람)

[Document(id='d358902a-63ec-4803-8dde-da7340843d57', metadata={'total_pages': 23}, page_content="<p id='95' data-category='paragraph' style='font-size:18px'>자료 : Bloomberg, 현대경제연구원.</p><p id='96' data-category='paragraph' style='font-size:22px'>¡ (시사점) 정부는 글로벌 자본시장에 대한 모니터링과 함께 자본시장 리스크를<br>지속해서 관리해야 하며, 투자자는 과도한 리스크 추구 행위를 지양해야 함</p><p id='97' data-category='paragraph' style='font-size:22px'>이 택 근 연 구 위 원 (2072-6366, t k l e e @ h r i . c o . k r )</p><p id='98' data-category='footnote' style='font-size:14px'>5) 채권지수(미 국채 10년, 신흥국 국채, 회사채), 주가지수(S&P500, MSCI 비미국·신흥국), 원자재(원유,<br>금, 은), 비트코인 등 11개 자산의 월간 수익률로 산출한 이동상관계수(3개월)를 3년 이동평균한 값.</p><footer id='99' style='font-size:14px'>10</footer><header id='100' style='font-size:16px'>2026년 글로벌 트렌드</header><h1 id='101' style='font-size:22px'>⑤ 우려되는 글로벌 중산층 소비 위축6)</h1><br><p id='102' data-category='paragraph' style='font-size:20px'>¡ (의미) 성장 둔화, 고물가, 고금리 등이 글로벌 중산층의 생계를 압박, 소비<br>둔화 등을 통해 세계 경제의 성장엔진 역할을 약화시킬 것으로 우려</p><br>"),
 Document(id='

In [17]:
# 2-2단계. 체인을 구성해서 LLM에게 필요한 답변을 얻는다.

llm = ChatUpstage(api_key="up_HIH5w7MHZN1dRIsK7A2GraLbJTG5d", model="solar-pro")
llm

ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x000002B4DE4E6D50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002B4DEAA63F0>, model_name='solar-pro', model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar')

In [18]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(
input_variables = ['query', 'context'],
template =
"""
## 역할
- 너는 세계 최고의 투자 분석 전문가이자, 친절한 교사야

## 목적
- 제공된 문서 내용을 기반으로, 유저의 질문에 답변을 하는 것 입니다.

## 유저 질문
{query}

## 문서 내용
{context}
"""
)

prompt_template

PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='\n## 역할\n- 너는 세계 최고의 투자 분석 전문가이자, 친절한 교사야\n\n## 목적\n- 제공된 문서 내용을 기반으로, 유저의 질문에 답변을 하는 것 입니다.\n\n## 유저 질문\n{query}\n\n## 문서 내용\n{context}\n')

In [19]:
from langchain_core.output_parsers import StrOutputParser

# 유저 질문
query = input('투자 트렌드에 궁금하신 점을 입력해주세요')

# 유저 질문을 기반으로 한 가장 유사한 문장 검색한 결과
context = retriever.invoke(query)
context_docs = context[0].page_content

chain = prompt_template |llm | StrOutputParser()

In [20]:
res = chain.invoke({'query': query, 'context': context_docs})
print(res)

현재 글로벌 경제 환경을 고려할 때, 투자 전략을 수립하기 위해 다음과 같은 분석과 접근 방식을 제안드립니다. 제공된 문서 내용과 글로벌 트렌드를 종합해 설명드리겠습니다.

---

### 1. **문서에서 도출한 핵심 리스크 요인**
   - **자본시장 변동성 증가**: 채권, 주식, 원자재, 암호화폐 등 주요 자산 간 상관관계가 높아지며 시스템적 리스크가 확대될 가능성이 있습니다(3개월 이동상관계수 3년 평균 기준).
   - **글로벌 중산층 소비 위축**: 고물가·고금리로 인해 중산층의 구매력이 감소하면서 세계 경제 성장이 둔화될 우려가 있습니다. 이는 소비재 산업(예: 자동차, 내구재, 고급 소비재)에 부정적 영향을 미칠 수 있습니다.

---

### 2. **추천 투자 전략**
#### (1) **방어적 자산 배분**
   - **채권**: 미국 국채(10년물) 등 안전자산 비중을 늘려 변동성 헤징. 신흥국 채권은 리스크 관리가 필요합니다.
   - **금(Gold)**: 인플레이션 헤지와 지리정치적 리스크 대비 수단으로 적합합니다.
   - **배당주**: S&P500 내 안정적인 배당 성장 기업(예: 유틸리티, 필수소비재)을 고려합니다.

#### (2) **성장 가능성 있는 섹터**
   - **AI/반도체**: 장기적인 기술 혁신 트렌드는 지속될 전망입니다. AI 인프라(예: NVIDIA, TSMC)와 클라우드 컴퓨팅 기업에 주목합니다.
   - **재생에너지**: 정부 지원 정책과 탄소 중산 목표로 태양광·풍력 관련 기업의 수혜가 예상됩니다.
   - **헬스케어**: 인구 고령화와 필수 의료 수요는 지속될 전망입니다.

#### (3) **신흥국 투자 시 주의점**
   - 문서는 신흥국 채권·주식의 높은 변동성을 경고합니다. 투자 시 **개별 국가의 재정 건전성**(예: 인도네시아, 베트남)과 **통화 안정성**을 꼼꼼히 검토해야 합니다.

#### (4) **과도한 리스크 회피**
   - 암호화폐(비트코인 등)와 레버리지 상품은 